## Library version check

In [1]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import math
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.5.2
numpy version: 1.21.6
matplotlib version: 3.5.2
tqdm version: 4.65.2
sktime version: 0.20.1
xgboost version: 1.7.2
seaborn version: 0.11.2
scikit-learn version: 1.1.3
------------------------------------------------------------------------------


## 0. load the libararies

In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.model_selection import StratifiedKFold , KFold
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 30)

## 1. preprocessing the data

In [3]:
train = pd.read_csv('./data/train.csv')

In [4]:
test = pd.read_csv('./data/test.csv')

In [5]:
## 변수들을 영문명으로 변경
cols = ['num_date_time', 'build_num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'isolation', 'sunshine', 'power']
train.columns = cols

def summer_cos(date):
    start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

    period = (end_date - start_date).total_seconds()

    return math.cos(2 * math.pi * (date - start_date).total_seconds() / period)

def summer_sin(date):
    start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

    period = (end_date - start_date).total_seconds()

    return math.sin(2 * math.pi * (date - start_date).total_seconds() / period)

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['date_time'] = pd.to_datetime(train['date_time'])
train['date'] = date.dt.date
train['hour'] = date.dt.hour
train['day'] = date.dt.day
train['weekday'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.isocalendar().week.astype(np.int32)

# 이상치 처리
train.loc[13238:13826, 'power'] += 3500
train.loc[19161:20343, 'power'] -= 4000
train = train.drop(index=range(114240, 114408))
train = train.reset_index(drop=True)

In [6]:
import pandas as pd

def copy_pattern_by_days(
    df,
    build_num,
    target_start,
    target_end,
    offset_days,          # 예: -7(이전 주), +7(다음 주), +3(3일 뒤) 등
    col='power',
    dt_col='date_time',
    inplace=False
):
    """
    [target_start ~ target_end] 구간의 값을
    (offset_days 만큼 이동한 구간)의 패턴으로 덮어쓰기.
    """
    _df = df if inplace else df.copy()

    ts, te = pd.to_datetime(target_start), pd.to_datetime(target_end)
    ss, se = ts + pd.Timedelta(days=offset_days), te + pd.Timedelta(days=offset_days)

    m_src = (_df['build_num'] == build_num) & (_df[dt_col] >= ss) & (_df[dt_col] <= se)
    m_tgt = (_df['build_num'] == build_num) & (_df[dt_col] >= ts) & (_df[dt_col] <= te)

    src_vals = _df.loc[m_src].sort_values(dt_col)[col].values
    tgt_idx  = _df.loc[m_tgt].sort_values(dt_col).index

    if len(src_vals) == 0 or len(tgt_idx) == 0:
        return _df  # 소스/타겟이 없으면 그대로 반환

    n = min(len(src_vals), len(tgt_idx))
    _df.loc[tgt_idx[:n], col] = src_vals[:n]
    return _df


def batch_copy_patterns_by_days(
    df,
    jobs,                 # [(build_num, t_start, t_end, offset_days), ...] 또는 dict 리스트
    col='power',
    dt_col='date_time',
    inplace=False,
    verbose=False
):
    """
    여러 건을 한 번에 처리하는 배치 함수.
    jobs 원소 형태:
      - 튜플: (build_num, target_start, target_end, offset_days)
      - 딕셔너리: {
            "build_num": ...,
            "target_start": ...,
            "target_end": ...,
            # 아래 중 하나
            "offset_days": ...,
            "week_offset": ...  # 있으면 7*week_offset으로 변환
        }
      ※ offset_days가 있으면 week_offset보다 우선
    """
    def _parse(job):
        if isinstance(job, (list, tuple)) and len(job) == 4:
            b, ts, te, od = job
            return b, ts, te, od
        if isinstance(job, dict):
            b  = job['build_num']
            ts = job['target_start']
            te = job['target_end']
            if 'offset_days' in job:
                od = job['offset_days']
            elif 'week_offset' in job:
                od = 7 * job['week_offset']
            else:
                raise ValueError("dict job에는 'offset_days' 또는 'week_offset' 중 하나가 필요합니다.")
            return b, ts, te, od
        raise ValueError("jobs 항목은 (build_num, start, end, offset_days) 튜플 또는 해당 키를 가진 dict여야 합니다.")

    _df = df if inplace else df.copy()

    for job in jobs:
        b, ts, te, od = _parse(job)
        if verbose:
            print(f"[batch] build_num={b}, target=({ts}~{te}), offset_days={od}")
        _df = copy_pattern_by_days(
            _df, b, ts, te, od, col=col, dt_col=dt_col, inplace=True
        )
    return _df


jobs = [
    (5, "2024-08-04 00:00", "2024-08-04 23:00", -7),
    (6, "2024-08-15 00:00", "2024-08-15 23:00", -4),
    (6, "2024-08-16 00:00", "2024-08-16 23:00", -7),
    (6, "2024-08-17 00:00", "2024-08-17 23:00", -7),
    (6, "2024-08-18 00:00", "2024-08-18 23:00", -7),
    (7, "2024-07-07 10:00", "2024-07-08 11:00", -7),
    (8,  "2024-07-21 08:00", "2024-07-21 11:00", -7),
    (8,  "2024-08-24 00:00", "2024-08-24 23:00", -7),
    (12, "2024-07-21 00:00", "2024-07-21 23:00", +7),
    (12, "2024-08-24 00:00", "2024-08-24 23:00", -7),
    (17, "2024-06-25 15:00", "2024-06-26 09:00", -7),
    (20, "2024-06-01 00:00", "2024-06-01 23:00", +7),
    (25, "2024-07-04 12:00", "2024-07-04 14:00", +7),
    (26, "2024-06-17 14:00", "2024-06-18 11:00", -7),
    (29, "2024-06-15 22:00", "2024-06-15 23:00", -7),
    (29, "2024-06-27 00:00", "2024-06-27 01:00", -7),
    (30, "2024-08-04 00:00", "2024-08-04 23:00", -1),
    (30, "2024-08-05 00:00", "2024-08-05 23:00", -1),
    (30, "2024-08-07 00:00", "2024-08-07 23:00", -1),
    (40, "2024-07-14 00:00", "2024-07-14 01:00", -1),
    (41, "2024-06-22 01:00", "2024-06-22 04:00", -7),
    (41, "2024-07-17 00:00", "2024-07-17 23:00", -7),
    (42, "2024-07-17 00:00", "2024-07-17 23:00", -1),
    (43, "2024-06-10 17:00", "2024-06-10 18:00", -7),
    (43, "2024-08-12 16:00", "2024-08-12 17:00", -7),
    (43, "2024-07-20 00:00", "2024-07-21 23:00", -7),
    (67, "2024-07-26 00:00", "2024-07-27 23:00", -7),
    (68, "2024-06-28 23:00", "2024-06-29 01:00", -7),
    (72, "2024-07-21 10:00", "2024-07-21 11:00", -7),
    (76, "2024-06-03 13:00", "2024-06-03 13:00", -1),
    (76, "2024-06-20 12:00", "2024-06-20 15:00", -7),
    (78, "2024-07-17 14:00", "2024-07-17 14:00", -7),
    (80, "2024-07-06 10:00", "2024-07-06 15:00", -7),
    (80, "2024-07-08 12:00", "2024-07-08 13:00", -7),
    (80, "2024-07-20 10:00", "2024-07-20 12:00", -7),
    (88, "2024-08-23 07:00", "2024-08-23 08:00", -7),
    (90, "2024-06-05 18:00", "2024-06-05 18:00", -1),
    (92, "2024-07-17 18:00", "2024-07-17 21:00", -7),
    (94, "2024-07-26 20:00", "2024-07-28 10:00", -7),
    (97, "2024-07-17 14:00", "2024-07-17 14:00", -7),
    (98, "2024-06-13 15:00", "2024-06-13 15:00", -7)
]

train = batch_copy_patterns_by_days(train, jobs, col='power', dt_col='date_time', inplace=False, verbose=False)

In [7]:
def batch_interpolate_building_power(df, targets, target_col='power', method='time'):
    """
    여러 건물/시점(단일 or 구간)에 대해 시간 기반 보간을 한 번에 적용하는 함수.

    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        targets (list of tuples): 
            [(build_num, start_time), (build_num, start_time, end_time), ...] 형태의 리스트
            - end_time이 없으면 단일 시점 처리
        target_col (str): 보간할 컬럼명
        method (str): pandas.interpolate method

    Returns:
        pd.DataFrame: 보간이 적용된 원본 데이터프레임
    """
    for item in targets:
        # 튜플 길이에 따라 단일 시점/구간 처리
        if len(item) == 2:
            build_num, start_time = item
            end_time = None
        elif len(item) == 3:
            build_num, start_time, end_time = item
        else:
            raise ValueError("targets는 (build_num, start_time) 또는 (build_num, start_time, end_time) 형식이어야 합니다.")

        # 대상 건물 데이터 추출
        building = df[df['build_num'] == build_num].sort_values('date_time').copy()

        # 결측 처리
        if end_time is None:
            mask_missing = (building['date_time'] == pd.Timestamp(start_time))
        else:
            mask_missing = (
                (building['date_time'] >= pd.Timestamp(start_time)) &
                (building['date_time'] <= pd.Timestamp(end_time))
            )
        building.loc[mask_missing, target_col] = np.nan

        # 시간 기반 보간
        building.set_index('date_time', inplace=True)
        building[target_col] = building[target_col].interpolate(method=method)
        building.reset_index(inplace=True)

        # 원본 반영
        df.loc[df['build_num'] == build_num, target_col] = building[target_col].values

    return df

targets = [
    (3, '2024-07-17 14:00'),
    (7, '2024-08-06 03:00'),
    (18, '2024-07-17 14:00'),
    (30, '2024-07-13 20:00'),
    (30, '2024-07-25 00:00'),
    (42, '2024-07-17 14:00'),
    (47, '2024-07-17 14:00'),
    (55, '2024-07-17 14:00'),
    (76, '2024-08-22 21:00'),
    (81, '2024-06-27 14:00'),
    (81, '2024-07-17 14:00'),
    (82, '2024-07-17 14:00'),
    (83, '2024-07-17 14:00'), 
    (5, '2024-08-04 06:00', '2024-08-04 08:00'), 
    (18, '2024-06-11 17:00', '2024-06-11 18:00'), 
    (18, '2024-08-08 15:00', '2024-08-08 16:00'), 
    (28, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (38, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (41, '2024-07-17 09:00', '2024-07-17 15:00'), 
    (60, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (62, '2024-07-17 13:00', '2024-07-17 15:00'), 
    (69, '2024-07-17 14:00', '2024-07-17 15:00'),  
    (76, '2024-06-20 12:00', '2024-06-20 16:00'),  
    (78, '2024-07-17 13:00', '2024-07-17 14:00'),
    (89, '2024-07-12 08:00', '2024-07-12 10:00'), 
    (97, '2024-07-17 13:00', '2024-07-17 15:00'), 
]

train = batch_interpolate_building_power(train, targets)

In [8]:
def batch_apply_pattern_scaling(df, tasks, target_col='power'):
    """
    여러 패턴 복사 작업을 한 번에 처리.
    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        tasks (list of tuples): 
            [
                (build_num, source_start, source_end, value_start, value_end, target_start, target_end),
                ...
            ]
        target_col (str): 수정할 컬럼명
    Returns:
        pd.DataFrame: 수정된 데이터프레임
    """
    for build_num, source_start, source_end, value_start_time, value_end_time, target_start, target_end in tasks:
        # 1. 원본 패턴 추출
        pattern_mask = (
            (df['build_num'] == build_num) &
            (df['date_time'] >= pd.Timestamp(source_start)) &
            (df['date_time'] <= pd.Timestamp(source_end))
        )
        P_source = df.loc[pattern_mask].sort_values('date_time')[target_col].values
        if len(P_source) == 0:
            continue  # 패턴 없으면 스킵

        # 2. 시작/종료 값
        V_start = df.loc[
            (df['build_num'] == build_num) & (df['date_time'] == pd.Timestamp(value_start_time)),
            target_col
        ].values[0]
        V_end = df.loc[
            (df['build_num'] == build_num) & (df['date_time'] == pd.Timestamp(value_end_time)),
            target_col
        ].values[0]

        # 3. 정규화 및 스케일링
        P_min, P_max = P_source.min(), P_source.max()
        P_scaled = (P_source - P_min) / (P_max - P_min + 1e-8)
        P_target = V_start + (V_end - V_start) * P_scaled

        # 4. 대상 구간 인덱스
        target_mask = (
            (df['build_num'] == build_num) &
            (df['date_time'] >= pd.Timestamp(target_start)) &
            (df['date_time'] <= pd.Timestamp(target_end))
        )
        target_indices = df.loc[target_mask].sort_values('date_time').index

        # 5. 길이 맞춰 삽입
        length = min(len(P_target), len(target_indices))
        df.loc[target_indices[:length], target_col] = P_target[:length]

    return df

tasks = [
    (7, '2024-06-30 10:00', '2024-07-01 11:00', '2024-07-07 09:00', '2024-07-08 12:00', '2024-07-07 10:00', '2024-07-08 11:00'),
    (7, '2024-07-05 14:00', '2024-07-05 23:00', '2024-07-12 13:00', '2024-07-13 00:00', '2024-07-12 14:00', '2024-07-12 23:00'),
]

train = batch_apply_pattern_scaling(train, tasks)

In [9]:
def batch_fill_hourly_means(df, tasks, target_col='power'):
    """
    여러 건물/기간/시간대 평균을 다른 날짜로 삽입하는 배치 함수.

    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        tasks (list of tuples): 
            [
                (build_num, source_dates, source_hours, target_date),
                ...
            ]
            - source_dates: ('start_date','end_date') or ['date1','date2',...]
            - source_hours: [hour1, hour2, ...]
            - target_date: 단일 날짜
        target_col (str): 수정할 컬럼명 (기본 'power')
    Returns:
        pd.DataFrame: 수정된 데이터프레임
    """
    for build_num, source_dates, source_hours, target_date in tasks:
        building = df[df['build_num'] == build_num].copy()

        # 날짜 마스크 생성
        if isinstance(source_dates, (tuple, list)) and len(source_dates) == 2 and not isinstance(source_dates[0], (pd.Timestamp, str)):
            # 범위일 경우
            start_date, end_date = pd.to_datetime(source_dates[0]).date(), pd.to_datetime(source_dates[1]).date()
            mask_range = (
                (building['date_time'].dt.date >= start_date) &
                (building['date_time'].dt.date <= end_date) &
                (building['date_time'].dt.hour.isin(source_hours))
            )
        else:
            # 날짜 리스트일 경우
            date_list = [pd.to_datetime(d).date() for d in source_dates]
            mask_range = (
                (building['date_time'].dt.date.isin(date_list)) &
                (building['date_time'].dt.hour.isin(source_hours))
            )

        # 시간별 평균 계산
        hourly_means = (
            building[mask_range]
            .groupby(building['date_time'].dt.hour)[target_col]
            .mean()
            .to_dict()
        )

        # 타겟 날짜에 삽입
        for hour, mean_val in hourly_means.items():
            mask_fill = (
                (df['build_num'] == build_num) &
                (df['date_time'].dt.date == pd.to_datetime(target_date).date()) &
                (df['date_time'].dt.hour == hour)
            )
            df.loc[mask_fill, target_col] = mean_val

    return df

tasks = [
    (67, ('2024-06-03', '2024-06-07'), [16, 17, 18], '2024-06-10'),          # 6/3~6/7 오후 4~6시 평균 → 6/10
    (67, ('2024-07-29', '2024-07-31'), [15, 16], '2024-08-01'),              # 7/29~7/31 오후 3~4시 평균 → 8/1
    (67, ['2024-08-13', '2024-08-14', '2024-08-16'], [16, 17], '2024-08-12'), # 8/13,14,16 오후 4~5시 평균 → 8/12
    (80, ('2024-07-01', '2024-07-05'), [11,12,13,14,19,20], '2024-07-08')    # 7/1~7/5 11~14시,19~20시 평균 → 7/8
]

train = batch_fill_hourly_means(train, tasks)

In [10]:
def scale_power_segments(train: pd.DataFrame, segments: list):
    """
    주어진 구간의 power 값을 앞뒤 하루 같은 시간대 min-max 스케일로 보정.
    보정된 값은 train['power']에 덮어씌움.

    Parameters:
        train (pd.DataFrame): 'build_num', 'date_time', 'power' 컬럼 포함 데이터프레임
        segments (list): [(build_num, start_datetime, end_datetime), ...] 형식의 튜플 리스트
    Returns:
        pd.DataFrame: 보정된 train 데이터프레임
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    
    for build_num, start_str, end_str in segments:
        target_start = pd.Timestamp(start_str)
        target_end = pd.Timestamp(end_str)

        # 대상 건물 데이터
        building_data = train[train['build_num'] == build_num].sort_values(by='date_time')
        target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
        
        # 참조 구간: 앞뒤 하루 동일 시간대
        ref_mask = (
            ((building_data['date_time'] >= target_start - pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end - pd.Timedelta(days=1))) |
            ((building_data['date_time'] >= target_start + pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end + pd.Timedelta(days=1)))
        )
        ref_data = building_data.loc[ref_mask, 'power']
        if ref_data.empty:
            continue  # 참조 데이터가 없으면 스킵

        ref_min, ref_max = ref_data.min(), ref_data.max()
        target_data = building_data.loc[target_mask, 'power']
        if target_data.empty or target_data.max() == target_data.min():
            continue  # 대상 데이터가 없거나 변동이 없으면 스킵

        # 스케일 조정
        scaled = (target_data - target_data.min()) / (target_data.max() - target_data.min())  # 0~1 정규화
        scaled = scaled * (ref_max - ref_min) + ref_min

        # train에 덮어쓰기
        train.loc[target_mask & (train['build_num'] == build_num), 'power'] = scaled

    return train

segments = [
    (30, '2024-06-20 06:00', '2024-06-20 23:00'),
    (30, '2024-07-06 06:00', '2024-07-06 23:00'),
]

train = scale_power_segments(train, segments)

In [11]:
def fill_with_weekly_pattern(train: pd.DataFrame, build_num: int, start_str: str, end_str: str):
    """
    특정 구간을 앞주+다음주 동일 시각 데이터 평균으로 채우고 train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time')

    # 대상 구간
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)
    target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
    target_range = building_data.loc[target_mask, ['date_time']].copy()
    if target_range.empty:
        print(f"⚠️ 대상 구간({start_str}~{end_str}) 데이터 없음")
        return train

    # 앞주 & 다음주 동일 시각 데이터 가져오기
    week_offset = pd.Timedelta(days=7)
    ref1 = building_data.set_index('date_time').loc[target_start - week_offset : target_end - week_offset, ['power']].reset_index()
    ref2 = building_data.set_index('date_time').loc[target_start + week_offset : target_end + week_offset, ['power']].reset_index()

    # 두 주 패턴 align (길이가 다를 경우 보정)
    if len(ref1) != len(target_range):
        ref1 = ref1.reindex(range(len(target_range)), method='nearest')
    if len(ref2) != len(target_range):
        ref2 = ref2.reindex(range(len(target_range)), method='nearest')

    # 두 주 평균 패턴 생성
    ref_mean = (ref1['power'].values + ref2['power'].values) / 2
    target_range['power_filled'] = ref_mean

    # 덮어쓰기
    for idx, row in target_range.iterrows():
        train.loc[
            (train['build_num'] == build_num) & (train['date_time'] == row['date_time']),
            'power'
        ] = row['power_filled']

    return train

# 7월 20일 02시 ~ 7월 22일 10시, 건물 49
train = fill_with_weekly_pattern(
    train, 
    build_num=43, 
    start_str="2024-07-20 02:00", 
    end_str="2024-07-22 10:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=53, 
    start_str="2024-06-14 16:00", 
    end_str="2024-06-17 09:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=67, 
    start_str="2024-07-27 00:00", 
    end_str="2024-07-28 00:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=94, 
    start_str="2024-07-27 00:00", 
    end_str="2024-07-28 00:00"
)

In [12]:
def add_value_to_segment(train: pd.DataFrame, build_num: int, start_str: str, end_str: str, add_value: float):
    """
    특정 건물의 지정 구간에 일정 값을 더해 train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)

    mask = (
        (train['build_num'] == build_num) &
        (train['date_time'] >= target_start) &
        (train['date_time'] <= target_end)
    )

    train.loc[mask, 'power'] = train.loc[mask, 'power'] + add_value
    return train

train = add_value_to_segment(
    train,
    build_num=53,
    start_str="2024-08-18 16:00",
    end_str="2024-08-19 07:00",
    add_value=400
)

train = add_value_to_segment(
    train,
    build_num=67,
    start_str="2024-06-01 00:00",
    end_str="2024-06-03 09:00",
    add_value=780
)

In [13]:
def fill_with_prev_next_day_avg(train: pd.DataFrame, build_num: int, start_str: str, end_str: str):
    """
    특정 건물의 지정 구간을 하루 전/하루 뒤 동일 시간대의 평균 값으로 채움.
    train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time')

    # 대상 구간
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)
    target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
    target_times = building_data.loc[target_mask, 'date_time']
    if target_times.empty:
        print(f"⚠️ 대상 구간({start_str}~{end_str}) 데이터 없음")
        return train

    # 하루 전/하루 뒤 동일 시간대 구간
    prev_day_mask = (building_data['date_time'] >= target_start - pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end - pd.Timedelta(days=1))
    next_day_mask = (building_data['date_time'] >= target_start + pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end + pd.Timedelta(days=1))
    prev_data = building_data.loc[prev_day_mask, ['date_time', 'power']]
    next_data = building_data.loc[next_day_mask, ['date_time', 'power']]

    if prev_data.empty and next_data.empty:
        print(f"⚠️ 참조 데이터 없음({start_str}~{end_str})")
        return train

    # 평균 패턴 계산 (있으면 합쳐서 평균)
    ref_values = []
    if not prev_data.empty:
        ref_values.append(prev_data['power'].values)
    if not next_data.empty:
        ref_values.append(next_data['power'].values)
    ref_mean = sum(ref_values) / len(ref_values)  # 두 날짜 평균

    # 길이가 다르면 맞춰서 채움
    ref_mean_series = pd.Series(ref_mean)
    ref_mean_series = ref_mean_series.reindex(range(len(target_times)), method='nearest')

    # 덮어쓰기
    train.loc[
        (train['build_num'] == build_num) & (train['date_time'] >= target_start) & (train['date_time'] <= target_end),
        'power'
    ] = ref_mean_series.values

    return train

train = fill_with_prev_next_day_avg(
    train,
    build_num=70,
    start_str="2024-06-04 09:00",
    end_str="2024-06-05 09:00"
)

In [14]:
def fill_power_with_holiday_pattern(train, build_num, date_ranges):
    """
    특정 건물의 지정 날짜 구간 전력 사용량을
    전후 holiday 패턴(같은 시간대 평균)으로 대체하는 함수.

    Parameters:
        train (pd.DataFrame): 전체 데이터프레임
        build_num (int): 건물 번호
        date_ranges (list of tuples): [(start_date, end_date), ...] 형식의 구간 리스트 (문자열 or Timestamp)
    Returns:
        pd.DataFrame: power가 덮어씌워진 원본 train DataFrame
    """
    # 데이터 정렬 및 시간 변환
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time').copy()
    building_data['hour'] = building_data['date_time'].dt.hour

    # holiday 날짜
    holiday_dates = building_data[building_data['holiday'] == 1]['date_time']

    for start_date, end_date in date_ranges:
        target_start = pd.Timestamp(start_date)
        target_end = pd.Timestamp(end_date)

        # 전후 holiday 추출
        prev_holiday = holiday_dates[holiday_dates < target_start].max()
        next_holiday = holiday_dates[holiday_dates > target_end].min()
        if pd.isna(prev_holiday) or pd.isna(next_holiday):
            continue  # holiday 없으면 skip

        # 전후 holiday 패턴
        prev_pattern = building_data[building_data['date_time'].dt.date == prev_holiday.date()]
        next_pattern = building_data[building_data['date_time'].dt.date == next_holiday.date()]
        holiday_pattern = (prev_pattern.groupby('hour')['power'].mean() +
                           next_pattern.groupby('hour')['power'].mean()) / 2

        # 대체
        target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
        building_data.loc[target_mask, 'power'] = building_data.loc[target_mask].apply(
            lambda row: holiday_pattern.loc[row['hour']] if row['hour'] in holiday_pattern.index else row['power'],
            axis=1
        )

    # train에 반영
    train.loc[building_data.index, 'power'] = building_data['power']
    return train

In [15]:
# isolation = pd.pivot_table(train, values = 'isolation', index = ['build_num', 'hour', 'month'], aggfunc = np.mean).reset_index()
# sunshine = pd.pivot_table(train, values = 'sunshine', index = ['build_num', 'hour', 'month'], aggfunc = np.mean).reset_index()

# 건물별, 요일별, 시간별 전력소비량 평균
power_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['target_mean_1'] = train.progress_apply(lambda x : power_mean_1.loc[(power_mean_1.build_num == x['build_num']) & (power_mean_1.hour == x['hour']) & (power_mean_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 요일별, 시간별 전력소비량 표준편차
power_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['target_std_1'] = train.progress_apply(lambda x : power_std_1.loc[(power_std_1.build_num == x['build_num']) & (power_std_1.hour == x['hour']) & (power_std_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

## 건물별, 요일별, 시간별 전력소비량 최소
power_min_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.min).reset_index()
tqdm.pandas()
train['target_min_1'] = train.progress_apply(lambda x : power_min_1.loc[(power_min_1.build_num == x['build_num']) & (power_min_1.hour == x['hour']) & (power_min_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

## 건물별, 요일별, 시간별 전력소비량 최대
power_max_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.max).reset_index()
tqdm.pandas()
train['target_max_1'] = train.progress_apply(lambda x : power_max_1.loc[(power_max_1.build_num == x['build_num']) & (power_max_1.hour == x['hour']) & (power_max_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 시간별 전력소비량 평균
hour_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean_1'] = train.progress_apply(lambda x : hour_mean_1.loc[(hour_mean_1.build_num == x['build_num']) & (hour_mean_1.hour == x['hour']) ,'power'].values[0], axis = 1)

# 건물별, 시간별 전력소비량 표준편차
hour_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std_1'] = train.progress_apply(lambda x : hour_std_1.loc[(hour_std_1.build_num == x['build_num']) & (hour_std_1.hour == x['hour']) ,'power'].values[0], axis = 1)

## 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['weekday']<5 else 1, axis = 1)
train.loc[('20240606'<=train.date_time)&(train.date_time<'20240607'),'holiday'] = 1
train.loc[('20240815'<=train.date_time)&(train.date_time<'20240816'),'holiday'] = 1

# 규칙 정의 함수
def apply_holiday_rules(row):
    bn = row['build_num']
    wd = row['weekday']
    day = row['day']
    week = (row['day'] - 1) // 7 + 1  # 몇째 주인지 계산

    # 📌 개별 규칙 적용
    if bn == 2:   # 상용: 토요일 쉼 → holiday = 1 if 토요일 else 0
        return 1 if wd == 5 else 0
    elif bn == 7:   # 건물기타: 일요일 쉼
        return 1 if wd == 6 else 0
    elif bn == 18:  # 백화점: 일요일 쉼
        return 1 if wd == 6 else 0
    elif bn == 25:  # 아파트: 토요일에 적게 씀 (평일로 간주)
        return 0
    elif bn == 26:  # 건물기타: 주말에 더 씀 → 평일로 간주
        return 0
    elif bn == 27:  # 백화점: 둘째, 넷째주 일요일 쉼
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 29:  # 백화점: 매달 10일 쉼
        return 1 if day == 10 else 0
    elif bn == 31:  # 아파트: 휴일 없음 → 평일 취급
        return 0
    elif bn == 32:  # 백화점: 둘째, 넷째주 월요일 쉼
        return 1 if wd == 0 and week in [2, 4] else 0
    elif bn == 34:  # 백화점: 휴일 없음 → 평일 취급
        return 0
    elif bn == 35:  # 전화국: 휴일 없음 -> 평일 취급
        return 0
    elif bn == 36:  # 전화국: 휴일 없음 -> 평일 취급
        return 1 if wd in [5, 6] else 0
    elif bn == 40:  # 백화점: 둘째, 넷째주 월요일 쉼
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 41:
        return 0
    elif bn == 54:
        return 0
    elif bn == 57:
        return 0
    elif bn == 58:
        return 0
    elif bn == 59:
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 61:
        return 0
    elif bn == 63:
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn in [97]:  # 토요일쉼
        return 1 if wd == 5 else 0
    elif bn in [1,4,9,10,11,19,28,30,33,45,65,70,71,73,74,76,77,78,79,82,84,85,88,89,91,92,93,95,96,98,99,100]:
        return 0
    else:
        # 기본 규칙 유지
        return row['holiday']

# 규칙 적용
train['holiday'] = train.apply(apply_holiday_rules, axis=1)

single_day_holidays = [
    (19, '2024-06-10'),
    (19, '2024-07-08'),
    (19, '2024-08-19'),
    (23, '2024-06-07'),
    (23, '2024-08-16'),
    (29, '2024-06-23'),
    (29, '2024-07-28'),
    (45, '2024-06-10'),
    (45, '2024-07-08'),
    (45, '2024-08-19'),
    (49, '2024-08-22'),
    (54, '2024-06-17'),
    (54, '2024-07-01'),
    (54, '2024-08-19'),
    (67, '2024-07-26'),
    (79, '2024-06-17'),
    (79, '2024-07-01'),
    (79, '2024-08-19'),
    (94, '2024-06-07'),
    (94, '2024-08-16'),
    (95, '2024-07-08'),
    (95, '2024-08-05'),
]

for build_num, date_str in single_day_holidays:
    target_date = pd.to_datetime(date_str).date()  # 날짜만 비교
    train.loc[
        (train['build_num'] == build_num) & 
        (train['date_time'].dt.date == target_date),
        'holiday'
    ] = 1

single_day_no_holiday = [
    (67, '2024-06-06')
]

for build_num, date_str in single_day_no_holiday:
    target_date = pd.to_datetime(date_str).date()  # 날짜만 비교
    train.loc[
        (train['build_num'] == build_num) & 
        (train['date_time'].dt.date == target_date),
        'holiday'
    ] = 0

date_ranges = [
    ('2024-07-26 00:00:00', '2024-07-26 23:59:59'),
]
train = fill_power_with_holiday_pattern(train, build_num=67, date_ranges=date_ranges)
    
# 건물별, 휴일별, 시간별 전력소비량 평균
power_holiday_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'holiday'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['holiday_mean_1'] = train.progress_apply(lambda x : power_holiday_mean_1.loc[(power_holiday_mean_1.build_num == x['build_num']) & (power_holiday_mean_1.hour == x['hour']) & (power_holiday_mean_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

# 건물별, 휴일별, 시간별 전력소비량 표준편차
power_holiday_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'holiday'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['holiday_std_1'] = train.progress_apply(lambda x : power_holiday_std_1.loc[(power_holiday_std_1.build_num == x['build_num']) & (power_holiday_std_1.hour == x['hour']) & (power_holiday_std_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)
    
## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_hour'] = np.sin(2*np.pi*train.hour/24)
train['cos_hour'] = np.cos(2*np.pi*train.hour/24)
train['sin_date'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['cos_date'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)
train['sin_month'] = -np.sin(2 * np.pi * train['month']/12)
train['cos_month'] = -np.cos(2 * np.pi * train['month']/12)
train['sin_weekday'] = -np.sin(2 * np.pi * (train['weekday']+1)/7)
train['cos_weekday'] = -np.cos(2 * np.pi * (train['weekday']+1)/7)

#summer_sin, cos
train['summer_sin'] = train['date_time'].apply(summer_sin)
train['summer_cos'] = train['date_time'].apply(summer_cos)

## 화씨 온도
train['temp_F'] = (train['temp'] * 9/5) + 32 

## 체감 온도
train['WC']=13.12+0.6215*train['temp']-13.947*train['wind']**0.16+0.486*train['temp']*train['wind']**0.16

## 불쾌 지수
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

train['dew_point'] = train['temp'] - (100 - train['hum']) / 5

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['build_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

## mean temperature
train = train.merge(train[train['hour']%3==0].groupby(['build_num','date'])['temp'].mean().reset_index().rename(columns = {'temp':'mean_temp'}), on = ['build_num','date'], how = 'left')

## min temperature
train = train.merge(train[train['hour']%3==0].groupby(['build_num','date'])['temp'].min().reset_index().rename(columns = {'temp':'min_temp'}), on = ['build_num','date'], how = 'left')

## max temperature
train = train.merge(train[train['hour']%3==0].groupby(['build_num','date'])['temp'].max().reset_index().rename(columns = {'temp':'max_temp'}), on = ['build_num','date'], how = 'left')

## mean windspeed
train = train.merge(train.groupby(['build_num','date'])['wind'].mean().reset_index().rename(columns = {'wind':'mean_wind'}), on = ['build_num','date'], how = 'left')

# ## min windspeed
# train = train.merge(train.groupby(['build_num','date'])['wind'].min().reset_index().rename(columns = {'wind':'min_wind'}), on = ['build_num','date'], how = 'left')

# ## max windspeed
# train = train.merge(train.groupby(['build_num','date'])['wind'].max().reset_index().rename(columns = {'wind':'max_wind'}), on = ['build_num','date'], how = 'left')

## mean humidity
train = train.merge(train.groupby(['build_num','date'])['hum'].mean().reset_index().rename(columns = {'hum':'mean_hum'}), on = ['build_num','date'], how = 'left')

# ## min humidity
# train = train.merge(train.groupby(['build_num','date'])['hum'].min().reset_index().rename(columns = {'hum':'min_hum'}), on = ['build_num','date'], how = 'left')

# ## max humidity
# train = train.merge(train.groupby(['build_num','date'])['hum'].max().reset_index().rename(columns = {'hum':'max_hum'}), on = ['build_num','date'], how = 'left')

## mean THI
train = train.merge(train.groupby(['build_num','date'])['THI'].mean().reset_index().rename(columns = {'THI':'mean_THI'}), on = ['build_num','date'], how = 'left')

## mean CDH
train = train.merge(train.groupby(['build_num','date'])['CDH'].mean().reset_index().rename(columns = {'CDH':'mean_CDH'}), on = ['build_num','date'], how = 'left')

## mean WC
train = train.merge(train.groupby(['build_num','date'])['WC'].mean().reset_index().rename(columns = {'WC':'mean_WC'}), on = ['build_num','date'], how = 'left')

## z-score
train['z_score'] = train['target_mean_1'] / train['target_std_1']

## temp_diff
train['temp_diff'] = train['max_temp'] - train['min_temp']

train.drop(columns=['date','day','isolation','sunshine'], inplace=True)
train.head()

100%|████████████████████████████████████████████████████████████████████████| 203832/203832 [01:52<00:00, 1819.77it/s]


,num_date_time,build_num,date_time,temp,prec,wind,hum,power,hour,weekday,month,week,target_mean_1,target_std_1,target_min_1,...,temp_F,WC,THI,dew_point,CDH,mean_temp,min_temp,max_temp,mean_wind,mean_hum,mean_THI,mean_CDH,mean_WC,z_score,temp_diff
0,1_20240601 00,1,2024-06-01 00:00:00,18.3,0.0,2.6,82.0,5794.80,0,5,6,22,5420.153077,477.394177,4832.85,...,64.94,18.605526,52.9016,14.7,-7.7,20.6125,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,11.353622,7.2
1,1_20240601 01,1,2024-06-01 01:00:00,18.3,0.0,2.7,82.0,5591.85,1,5,6,22,5255.363077,498.582392,4391.73,...,64.94,18.569864,52.9016,14.7,-15.4,20.6125,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,10.540611,7.2
2,1_20240601 02,1,2024-06-01 02:00:00,18.1,0.0,2.6,80.0,5338.17,2,5,6,22,4954.530000,535.473239,3716.73,...,64.58,18.367969,51.6000,14.1,-23.3,20.6125,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,9.252619,7.2
3,1_20240601 03,1,2024-06-01 03:00:00,18.0,0.0,2.6,81.0,4554.42,3,5,6,22,4643.097692,652.249023,3073.47,...,64.40,18.249191,51.8809,14.2,-31.3,20.6125,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,7.118597,7.2
4,1_20240601 04,1,2024-06-01 04:00:00,17.8,0.0,1.3,81.0,3602.25,4,5,6,22,4645.227692,590.362239,3399.63,...,64.04,18.659442,51.5209,14.0,-39.5,20.6125,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,7.868436,7.2


In [16]:
#전력소비량 이상치 처리(건물번호를 기준)
for num in tqdm(range(train['build_num'].nunique())):
    train.loc[train['build_num'] == num+1, 'power'] = train.loc[train['build_num'] == num+1, 'power'].clip(train.loc[train['build_num'] == num+1, 'power'].quantile(.01), train.loc[train['build_num'] == num+1, 'power'].quantile(.99))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 181.48it/s]


## 2. Model : XGBoost

In [17]:
# Define SMAPE loss function
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 200

# Define the custom metric
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    numerator = np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))
    return np.mean(numerator) * 200

## 3. model tuning

## 4. test inference

### preprocessing for test data

In [18]:
# train set과 동일한 전처리 과정
test['일시'] = pd.to_datetime(test['일시'])
cols = ['num_date_time', 'build_num', 'date_time', 'temp' , 'prec', 'wind', 'hum']
test.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(test.date_time)
test['date'] = date.dt.date
test['hour'] = date.dt.hour
test['day'] = date.dt.day
test['weekday'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.isocalendar().week.astype(np.int32)

test['holiday'] = test.apply(lambda x : 0 if x['weekday']<5 else 1, axis = 1)
test['holiday'] = test.apply(apply_holiday_rules, axis=1)

# ## 일조
# tqdm.pandas()
# test['isolation'] = np.round(test.progress_apply(lambda x : isolation.loc[(isolation.build_num == x['build_num']) & (isolation.hour == x['hour']) & (isolation.month == x['month']) ,'isolation'].values[0], axis = 1), 1)

# ## 일사
# tqdm.pandas()
# test['sunshine'] = np.round(test.progress_apply(lambda x : sunshine.loc[(sunshine.build_num == x['build_num']) & (sunshine.hour == x['hour']) & (sunshine.month == x['month']) ,'sunshine'].values[0], axis = 1), 2)

# 건물별, 요일별, 시간별 전력소비량 평균
tqdm.pandas()
test['target_mean_1'] = test.progress_apply(lambda x : power_mean_1.loc[(power_mean_1.build_num == x['build_num']) & (power_mean_1.hour == x['hour']) & (power_mean_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 요일별, 시간별 전력소비량 표준편차
tqdm.pandas()
test['target_std_1'] = test.progress_apply(lambda x : power_std_1.loc[(power_std_1.build_num == x['build_num']) & (power_std_1.hour == x['hour']) & (power_std_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['target_min_1'] = test.progress_apply(lambda x : power_min_1.loc[(power_min_1.build_num == x['build_num']) & (power_min_1.hour == x['hour']) & (power_min_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['target_max_1'] = test.progress_apply(lambda x : power_max_1.loc[(power_max_1.build_num == x['build_num']) & (power_max_1.hour == x['hour']) & (power_max_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['holiday_mean_1'] = test.progress_apply(lambda x : power_holiday_mean_1.loc[(power_holiday_mean_1.build_num == x['build_num']) & (power_holiday_mean_1.hour == x['hour']) & (power_holiday_mean_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['holiday_std_1'] = test.progress_apply(lambda x : power_holiday_std_1.loc[(power_holiday_std_1.build_num == x['build_num']) & (power_holiday_std_1.hour == x['hour']) & (power_holiday_std_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_mean_1'] = test.progress_apply(lambda x : hour_mean_1.loc[(hour_mean_1.build_num == x['build_num']) & (hour_mean_1.hour == x['hour']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_std_1'] = test.progress_apply(lambda x : hour_std_1.loc[(hour_std_1.build_num == x['build_num']) & (hour_std_1.hour == x['hour']) ,'power'].values[0], axis = 1)

test['sin_hour'] = np.sin(2*np.pi*test.hour/24)
test['cos_hour'] = np.cos(2*np.pi*test.hour/24)
test['sin_date'] = -np.sin(2 * np.pi * (test['month']+test['day']/31)/12)
test['cos_date'] = -np.cos(2 * np.pi * (test['month']+test['day']/31)/12)
test['sin_month'] = -np.sin(2 * np.pi * test['month']/12)
test['cos_month'] = -np.cos(2 * np.pi * test['month']/12)
test['sin_weekday'] = -np.sin(2 * np.pi * (test['weekday']+1)/7.0)
test['cos_weekday'] = -np.cos(2 * np.pi * (test['weekday']+1)/7.0)

#summer_sin, cos
test['summer_sin'] = test['date_time'].apply(summer_sin)
test['summer_cos'] = test['date_time'].apply(summer_cos)

## 화씨 온도
test['temp_F'] = (test['temp'] * 9/5) + 32 

## 체감 온도
test['WC']=13.12+0.6215*test['temp']-13.947*test['wind']**0.16+0.486*test['temp']*test['wind']**0.16

## 불쾌 지수
test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

test['dew_point'] = test['temp'] - (100 - test['hum']) / 5

cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['build_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

## mean temperature
test = test.merge(test[test['hour']%3==0].groupby(['build_num','date'])['temp'].mean().reset_index().rename(columns = {'temp':'mean_temp'}), on = ['build_num','date'], how = 'left')

## min temperature
test = test.merge(test[test['hour']%3==0].groupby(['build_num','date'])['temp'].min().reset_index().rename(columns = {'temp':'min_temp'}), on = ['build_num','date'], how = 'left')

## max temperature
test = test.merge(test[test['hour']%3==0].groupby(['build_num','date'])['temp'].max().reset_index().rename(columns = {'temp':'max_temp'}), on = ['build_num','date'], how = 'left')

## mean windspeed
test = test.merge(test.groupby(['build_num','date'])['wind'].mean().reset_index().rename(columns = {'wind':'mean_wind'}), on = ['build_num','date'], how = 'left')

# ## min windspeed
# test = test.merge(test.groupby(['build_num','date'])['wind'].min().reset_index().rename(columns = {'wind':'min_wind'}), on = ['build_num','date'], how = 'left')

# ## max windspeed
# test = test.merge(test.groupby(['build_num','date'])['wind'].max().reset_index().rename(columns = {'wind':'max_wind'}), on = ['build_num','date'], how = 'left')

## mean humidity
test = test.merge(test.groupby(['build_num','date'])['hum'].mean().reset_index().rename(columns = {'hum':'mean_hum'}), on = ['build_num','date'], how = 'left')

# ## min humidity
# test = test.merge(test.groupby(['build_num','date'])['hum'].min().reset_index().rename(columns = {'hum':'min_hum'}), on = ['build_num','date'], how = 'left')

# ## max humidity
# test = test.merge(test.groupby(['build_num','date'])['hum'].max().reset_index().rename(columns = {'hum':'max_hum'}), on = ['build_num','date'], how = 'left')

## mean THI
test = test.merge(test.groupby(['build_num','date'])['THI'].mean().reset_index().rename(columns = {'THI':'mean_THI'}), on = ['build_num','date'], how = 'left')

## mean CDH
test = test.merge(test.groupby(['build_num','date'])['CDH'].mean().reset_index().rename(columns = {'CDH':'mean_CDH'}), on = ['build_num','date'], how = 'left')

## mean WC
test = test.merge(test.groupby(['build_num','date'])['WC'].mean().reset_index().rename(columns = {'WC':'mean_WC'}), on = ['build_num','date'], how = 'left')

## z-score
test['z_score'] = test['target_mean_1'] / test['target_std_1']

## temp_diff
test['temp_diff'] = test['max_temp'] - test['min_temp']

test.drop(['date_time','date','day'], axis = 1, inplace = True)
test.head()

100%|██████████████████████████████████████████████████████████████████████████| 16800/16800 [00:06<00:00, 2545.18it/s]


,num_date_time,build_num,temp,prec,wind,hum,hour,weekday,month,week,holiday,target_mean_1,target_std_1,target_min_1,target_max_1,...,temp_F,WC,THI,dew_point,CDH,mean_temp,min_temp,max_temp,mean_wind,mean_hum,mean_THI,mean_CDH,mean_WC,z_score,temp_diff
0,1_20240825 00,1,26.5,0.0,0.7,80.0,0,6,8,34,0,4688.3600,636.533183,3870.42,5924.79,...,79.70,28.580992,66.7200,22.5,0.5,28.3375,25.0,31.7,1.458333,76.25,69.47355,18.695833,30.722386,7.365460,6.7
1,1_20240825 01,1,26.1,0.0,0.0,80.0,1,6,8,34,0,4619.7050,805.525047,2967.93,5733.45,...,78.98,29.341150,66.0000,22.1,0.6,28.3375,25.0,31.7,1.458333,76.25,69.47355,18.695833,30.722386,5.735023,6.7
2,1_20240825 02,1,25.9,0.0,0.3,83.0,2,6,8,34,0,4571.3075,735.786140,3648.75,5689.17,...,78.62,28.095476,67.0821,22.5,0.5,28.3375,25.0,31.7,1.458333,76.25,69.47355,18.695833,30.722386,6.212821,6.7
3,1_20240825 03,1,25.7,0.0,1.1,83.0,3,6,8,34,0,4258.5975,738.620400,3187.41,5579.04,...,78.26,27.613364,66.7221,22.3,0.2,28.3375,25.0,31.7,1.458333,76.25,69.47355,18.695833,30.722386,5.765610,6.7
4,1_20240825 04,1,25.5,0.0,1.0,86.0,4,6,8,34,0,3920.8350,709.756021,1870.44,4618.41,...,77.90,27.414250,67.9824,22.7,-0.3,28.3375,25.0,31.7,1.458333,76.25,69.47355,18.695833,30.722386,5.524201,6.7


### KFold ensemble
#### - 모델의 일반화 성능개선을 위해 Out-Of-KFold ensemble을 사용하였습니다.

In [19]:
#범주형 변수설정
# train['hour'] = train['hour'].astype('category')
# train['month'] = train['month'].astype('category')
# train['weekday'] = train['weekday'].astype('category')
# train['holiday'] = train['holiday'].astype('category')

# test['hour'] = test['hour'].astype('category')
# test['month'] = test['month'].astype('category')
# test['weekday'] = test['weekday'].astype('category')
# test['holiday'] = test['holiday'].astype('category')

In [20]:
n_splits = 10
cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from xgboost import XGBRegressor

seeds = [42, 43, 44, 45, 46]

preds = np.array([])
cv_smape_per_build = []    
cv_smape_map = {}                 

for i in tqdm(range(100)):
    bnum = i + 1

    y_train = train.loc[train.build_num == bnum, 'power'].reset_index(drop=True)
    x_train = train.loc[train.build_num == bnum].drop(
        columns=['num_date_time','build_num','date_time','power']
    ).reset_index(drop=True)
    x_test = test.loc[test.build_num == bnum].drop(columns=['num_date_time','build_num'])
    x_test = x_test[x_train.columns]

    pred_df = pd.DataFrame()
    fold = 0
    fold_smapes = []

    for tri, vai in cv.split(x_train):
        
        val_preds_seeds = []
        test_preds_seeds = []

        for sd in seeds:
            xgb = XGBRegressor(
                n_estimators=5000, eta=0.1, min_child_weight=6,
                max_depth=12, colsample_bytree=0.8, subsample=0.9,
                seed=sd, eval_metric=smape,
                tree_method="gpu_hist", enable_categorical=True
            )

            xgb.fit(
                x_train.iloc[tri], y_train.iloc[tri],
                eval_set=[(x_train.iloc[vai], y_train.iloc[vai])],
                early_stopping_rounds=50, verbose=False
            )

            val_preds_seeds.append(xgb.predict(x_train.iloc[vai]))
            test_preds_seeds.append(xgb.predict(x_test))

        # --- seed 평균(앙상블) ---
        y_valid_pred_ens = np.mean(np.vstack(val_preds_seeds), axis=0)
        y_pred_test_ens  = np.mean(np.vstack(test_preds_seeds), axis=0)

        fold_smapes.append(SMAPE(y_train.iloc[vai].values, y_valid_pred_ens))

        pred_df.loc[:, fold] = y_pred_test_ens
        fold += 1

    # 빌딩별 CV 평균 SMAPE
    mean_cv_smape = float(np.mean(fold_smapes)) if len(fold_smapes) else np.nan
    cv_smape_per_build.append((bnum, mean_cv_smape))
    cv_smape_map[bnum] = mean_cv_smape
    print(f"[build {bnum:3d}] CV mean SMAPE (seed-ensemble): {mean_cv_smape:.4f}")

    pred = pred_df.mean(axis=1).values
    preds = np.append(preds, pred)

cv_smape_df = pd.DataFrame(cv_smape_per_build, columns=['build_num', 'cv_smape']).sort_values('build_num')
print("\n=== Per-build CV mean SMAPE (seed-ensemble) ===")
print(cv_smape_df)

overall_cv_smape = cv_smape_df['cv_smape'].mean()
print(f"\n=== Overall (macro) mean SMAPE across build_num (seed-ensemble): {overall_cv_smape:.4f} ===")

  1%|▊                                                                              | 1/100 [02:43<4:29:24, 163.28s/it]

[build   1] CV mean SMAPE (seed-ensemble): 6.5129


  2%|█▌                                                                             | 2/100 [04:58<3:59:43, 146.77s/it]

[build   2] CV mean SMAPE (seed-ensemble): 4.2068


  3%|██▎                                                                            | 3/100 [08:20<4:38:21, 172.18s/it]

...

[build  98] CV mean SMAPE (seed-ensemble): 3.3144


 99%|█████████████████████████████████████████████████████████████████████████████▏| 99/100 [4:40:07<02:43, 163.79s/it]

[build  99] CV mean SMAPE (seed-ensemble): 1.7696


100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [4:43:28<00:00, 170.09s/it]

[build 100] CV mean SMAPE (seed-ensemble): 7.1636

=== Per-build CV mean SMAPE (seed-ensemble) ===
    build_num  cv_smape
0           1  6.512944
1           2  4.206775
2           3  1.542018
3           4  3.862864
4           5  0.942368
..        ...       ...
95         96  3.207339
96         97  4.964964
97         98  3.314403
98         99  1.769552
99        100  7.163566

[100 rows x 2 columns]

=== Overall (macro) mean SMAPE across build_num (seed-ensemble): 2.5794 ===


In [24]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = preds
submission.to_csv('./submission/XGB_byb_final.csv', index = False)